# Capitulo 01 Introducao Sql Duckdb

Notebook gerado automaticamente a partir do código fonte python.


In [ ]:
# -*- coding: utf-8 -*-
"""
capitulo_01_introducao_sql_duckdb
"""

# capitulo_01_introducao_sql_duckdb
import duckdb
import os

In [ ]:
con.execute("""
-- CUIDADO: Isso remove TODAS as linhas!
DELETE FROM weather;
""")
print(con.fetchall())

### ⚠️ Cuidado com DELETE sem WHERE

In [ ]:
con.execute("""
-- Deletar linhas específicas
DELETE FROM weather WHERE city = 'Hayward';
""")
print(con.fetchall())

## 15. Exclusão de Dados (DELETE)

Removendo registros das tabelas.

In [ ]:
con.execute("""
-- Atualizar múltiplas colunas
UPDATE weather
SET temp_hi = temp_hi - 2, temp_lo = temp_lo - 2
WHERE date > '1994-11-28';
""")
print(con.fetchall())

## 14. Atualização de Dados (UPDATE)

Modificando registros existentes.

In [ ]:
con.execute("""
-- Cidades que começam com 'S'
SELECT city, max(temp_lo) FROM weather
WHERE city LIKE 'S%'
GROUP BY city
HAVING max(temp_lo) < 40;
""")
print(con.fetchall())

### Combinando WHERE e HAVING

In [ ]:
con.execute("""
-- Filtrar grupos
SELECT city, max(temp_lo) FROM weather
GROUP BY city
HAVING max(temp_lo) < 40;
""")
print(con.fetchall())

## 13. Filtros em Grupos (HAVING)

Filtrando resultados após agrupamento.

In [ ]:
con.execute("""
-- Agrupar por cidade
SELECT city, max(temp_lo) FROM weather GROUP BY city;
""")
print(con.fetchall())

## 12. Agrupamento (GROUP BY)

Agrupando dados para cálculos agregados.

In [ ]:
con.execute("""
-- Encontrar o valor máximo
SELECT max(temp_lo) FROM weather;

-- Encontrar a cidade com a temperatura mínima mais alta
SELECT city FROM weather
WHERE temp_lo = (SELECT max(temp_lo) FROM weather);
""")
print(con.fetchall())

## 11. Funções de Agregação

Funções para calcular valores agregados (MAX, MIN, AVG, etc.).

In [ ]:
con.execute("""
SELECT * FROM weather
LEFT OUTER JOIN cities ON weather.city = cities.name;
""")
print(con.fetchall())

### LEFT OUTER JOIN

In [ ]:
con.execute("""
SELECT * FROM weather
INNER JOIN cities ON weather.city = cities.name;
""")
print(con.fetchall())

### INNER JOIN

In [ ]:
con.execute("""
-- Junção implícita
SELECT * FROM weather, cities
WHERE city = name;

-- Especificando colunas
SELECT city, temp_lo, temp_hi, prcp, date, lon, lat
FROM weather, cities
WHERE city = name;
""")
print(con.fetchall())

## 10. Junções de Tabelas (JOINS)

Combinando dados de múltiplas tabelas.

In [ ]:
con.execute("""
-- Obter cidades únicas
SELECT DISTINCT city FROM weather;

-- DISTINCT com ordenação
SELECT DISTINCT city FROM weather ORDER BY city;
""")
print(con.fetchall())

## 9. Valores Únicos (DISTINCT)

Removendo duplicatas nos resultados.

In [ ]:
con.execute("""
-- Ordenar por uma coluna
SELECT * FROM weather ORDER BY city;

-- Ordenar por múltiplas colunas
SELECT * FROM weather ORDER BY city, temp_lo;
""")
print(con.fetchall())

## 8. Ordenação (ORDER BY)

Ordenando resultados.

In [ ]:
con.execute("""
-- Filtro simples
SELECT * FROM weather
WHERE city = 'San Francisco' AND prcp > 0.0;
""")
print(con.fetchall())

## 7. Filtros com WHERE

Filtrando resultados com condições.

In [ ]:
con.execute("""
-- Calcular temperatura média
SELECT city, (temp_hi + temp_lo) / 2 AS temp_avg, date
FROM weather;
""")
print(con.fetchall())

## 6. Expressões e Cálculos

Realizando cálculos em queries.

In [ ]:
con.execute("""
-- Selecionar todas as colunas
SELECT * FROM weather;

-- Selecionar colunas específicas
SELECT city, temp_lo, temp_hi, prcp, date FROM weather;
""")
print(con.fetchall())

## 5. Consultas Básicas (SELECT)

Selecionando dados das tabelas.

In [ ]:
con.execute("""
-- Inserção básica
INSERT INTO weather VALUES ('San Francisco', 46, 50, 0.25, '1994-11-27');

-- Inserção com colunas explícitas
INSERT INTO weather (city, temp_lo, temp_hi, prcp, date)
VALUES ('San Francisco', 43, 57, 0.0, '1994-11-29');

-- Inserção omitindo colunas (valores serão NULL)
INSERT INTO weather (date, city, temp_hi, temp_lo)
VALUES ('1994-11-29', 'Hayward', 54, 37);
""")
print(con.fetchall())

## 4. Inserção de Dados (INSERT)

Diferentes formas de inserir dados nas tabelas.

In [ ]:
# Conectar ao banco em memória
import duckdb
con = duckdb.connect(database=':memory:')

# Criar tabelas
con.execute("""
CREATE TABLE weather (
    city VARCHAR,
    temp_lo INTEGER,      -- temperatura mínima
    temp_hi INTEGER,      -- temperatura máxima
    prcp FLOAT,
    date DATE
);

CREATE TABLE cities (
    name VARCHAR,
    lat DECIMAL,
    lon DECIMAL
);
""")
print(con.fetchall())

## 3. Criação de Tabelas e Estruturas

Criando tabelas de exemplo para demonstrar operações SQL.

## 2. Imports e Configuração Inicial

Importando bibliotecas e criando conexão em memória.

In [ ]:
# Instalação de dependências
%pip install duckdb -q

## 1. Instalação de Dependências

Instalando o DuckDB para começar.